In [10]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import datetime
import time


In [11]:
#function to login into rapsodo, where practice sessions are loggged

def login(email, password):

    # Initialize Chrome webdriver 
    #driver = webdriver.Chrome()

    # Open the website
    driver.get('https://golf-cloud.rapsodo.com/')
    #wait = WebDriverWait(driver, 15)

    # Wait for the email input field to be visible
    email_input = wait.until(EC.presence_of_element_located((By.ID, 'login-email')))
    # Input email
    email_input.send_keys(email)

    # Input password
    password_input = wait.until(EC.presence_of_element_located((By.ID, 'login-password')))
    password_input.send_keys(password)

    # Click submit button
    submit_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="login-button"]')))
    submit_button.click()
    
    print("Login successful")
    
    #click onto sessions icon
    try:
        click_sessions = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div[1]/div/div[1]/div[2]/div/a[2]/div/div[1]'))).click()
        #sessions_button.click()
        print('Clicked on Sessions')

    except:
        print('Either element was not found, or Bot could not click on it.')
        pass
    
    ###sessions table###
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "session-page")))
    
    
    #while(True):
     #   pass
    #return driver


In [12]:
#function to find elements 
def find_session_rows():
    session_rows = driver.find_elements(By.CSS_SELECTOR, "div.sc-fLseNd.goIziV.rdt_TableBody > div")
    return session_rows


In [14]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
email = '********'
password = '*********'
login(email, password)

target_date = datetime.datetime(2024, 5, 31)
session_page = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.sc-hzhKNl.jgilmI.rdt_TableBody")))
elements = driver.find_elements(By.CSS_SELECTOR, 'div.sc-hzhKNl.jgilmI.rdt_TableBody > div')
dates =[]

#root > div.content > div > div > div.data-table-container.session > div > div.data-table-container.session > div > div > div > div.sc-hzhKNl.jgilmI.rdt_TableBody
for i in range(len(elements)):
    time.sleep(2)
    elements = find_session_rows()
    #print(elements[i])
    date_text = elements[i].find_element(By.CSS_SELECTOR, 'time').text
    #print(date_text)
    date = datetime.datetime.strptime(date_text, "%m/%d/%y")
    dates.append(date)
    
    if date > target_date:
        view_link = elements[i].find_element(By.CLASS_NAME, 'table-link-container')
        #print(view_link)
        view_link.click()
        print(f'{date} link clicked')
        
        #export csv
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="export-button"]')))
        driver.maximize_window()

        export_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="export-button"]')))
        export_button.click()
        #driver.execute_script("arguments[0].click();", export_button)
        print(f' {date} csv exported')
        time.sleep(10)
        driver.back()
        time.sleep(3)


Login successful
Either element was not found, or Bot could not click on it.


IndexError: list index out of range

In [15]:

### retrieving recent downloads and moving to desktop###
import os
import shutil
from pathlib import Path

# Path to the Downloads folder and Desktop folder
downloads_folder = Path("~/Downloads").expanduser()
desktop_folder = Path("~/Desktop").expanduser()

# Get a list of all files in the Downloads folder
files = [f for f in downloads_folder.iterdir() if f.is_file()]

# Sort files by modification time (recent first)
files.sort(key=lambda x: x.stat().st_mtime, reverse=True)

# Take the last x recent files
recent_files = files[:31]

# Move each recent file to the Desktop folder
for file in recent_files:
    shutil.move(str(file), str(desktop_folder / file.name))
    print(f"Moved {file.name} to Desktop.")


Moved mlm2pro_shotexport_072424.csv to Desktop.
Moved mlm2pro_shotexport_062724.csv to Desktop.
Moved mlm2pro_shotexport_062524.csv to Desktop.
Moved mlm2pro_shotexport_062124.csv to Desktop.
Moved mlm2pro_shotexport_062024.csv to Desktop.
Moved mlm2pro_shotexport_061924.csv to Desktop.
Moved mlm2pro_shotexport_060624 (1).csv to Desktop.
Moved mlm2pro_shotexport_060624.csv to Desktop.
Moved mlm2pro_shotexport_053124.csv to Desktop.
Moved mlm2pro_shotexport_042724 (7).csv to Desktop.
Moved mlm2pro_shotexport_042424.csv to Desktop.
Moved mlm2pro_shotexport_041824 (1).csv to Desktop.
Moved mlm2pro_shotexport_041024 (3).csv to Desktop.
Moved mlm2pro_shotexport_040524 (1).csv to Desktop.
Moved mlm2pro_shotexport_031524 (1).csv to Desktop.
Moved mlm2pro_shotexport_030524 (3).csv to Desktop.
Moved mlm2pro_shotexport_030524 (2).csv to Desktop.
Moved mlm2pro_shotexport_030124 (1).csv to Desktop.
Moved mlm2pro_shotexport_022924 (1).csv to Desktop.
Moved mlm2pro_shotexport_022724 (3).csv to Deskt

In [16]:
#function to add shot type
def add_shot(x):
    if x['Launch Direction'] > -1.5 and x['Launch Direction'] < 1.5: 
        return 'Straight'
    elif x['Launch Direction'] < -1.5: 
        return 'Left'
    else: 
        return 'Right'

In [17]:
#function to clean DFs
def clean_pd(df):
    clean = df.copy()
    clean = clean.dropna()
    clean = clean.drop(clean[clean.iloc[:, 0] == 'Club Type'].index).reset_index(drop = True)
    clean['Launch Direction'] = clean['Launch Direction'].astype(float)
    clean['Shot Type'] = clean.apply(add_shot, axis =1)
    return clean 


In [18]:
import pandas as pd
#### merging recently downloaded sessions into master file###

# Define paths to Desktop folder and master file
desktop_folder = Path("~/Desktop").expanduser()
master_file = desktop_folder / "Golf Stats Master.csv"

# Get a list of all CSV files on the Desktop
csv_files = [f for f in desktop_folder.iterdir() if f.is_file() and f.suffix == ".csv"]

# Sort CSV files by modification time (recent first)
csv_files.sort(key=lambda x: x.stat().st_mtime, reverse=True)

# Take the last x recent CSV files
recent_csv_files = csv_files[:31]

# Clean and merge recent CSV files into the master file
if master_file.exists():
    # Read the master file
    master_df = pd.read_csv(master_file)
else:
    # Create an empty DataFrame if master file doesn't exist
    master_df = pd.DataFrame()

# Apply the clean_df function to each recent CSV file
for csv_file in recent_csv_files:
    df = pd.read_csv(csv_file, header = 1)
    #print(df.columns)
    df = clean_pd(df)  # Apply cleaning function
    master_df = pd.concat([master_df, df], ignore_index=True)

# Write the merged data to the master file, overwriting it
master_df.to_csv(master_file, index=False)

print(f"Merged {len(recent_csv_files)} recent CSV files after cleaning into {master_file}.")


Merged 31 recent CSV files after cleaning into /Users/margiselleestevez/Desktop/Golf Stats Master.csv.
